In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_rel, wilcoxon
from sklearn import datasets, preprocessing
from sklearn.model_selection import cross_val_predict, cross_val_score, RepeatedStratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils import resample

from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y
from collections import Counter
import time
import random
import math
from sklearn.metrics import accuracy_score

In [ ]:
#carrega a base wine
wine = datasets.load_wine()
wine_X = wine.data
wine_y = wine.target

In [ ]:
#definido o classificador heterogeneouspooling com hill climbing
class HeterogenousPoolingClassifier_Heuristico(BaseEstimator):
    def __init__(self, heuristica=None, n_samples=None):
        super().__init__()
        self.heuristica = heuristica
        self.n_samples = n_samples #apenas um hiperparametro
    
    #método fit() para treinamento dos classificadores base    
    def fit(self,x_train,y_train):
        
        classificadores = [] #array classifiacdores: array para salvar os classificadores base depois de treinados
        classificadores_optimal = []
        x_train,y_train = check_X_y(x_train,y_train)
        
        #_ordem_classes: array com as classes da base de treino original, em ordem decrescente(utlizado para desempate)
        df_aux = pd.DataFrame(data=y_train)
        df_ordem = df_aux.value_counts(df_aux[0])
        dados_ordem = pd.DataFrame(data=df_ordem)
        self._ordem_classes  = dados_ordem.index.array
        
        #loop para criar novas bases de treino a partir da original, e ir fitando os classifiacdores base
        for i in range(self.n_samples):
            sub_X, sub_y = resample(x_train, y_train, random_state=i)


            self.DT = DecisionTreeClassifier()
            self.DT.fit(sub_X,sub_y)
            classificadores.append(self.DT)
            
            self.gNB = GaussianNB()
            self.gNB.fit(sub_X,sub_y)
            classificadores.append(self.gNB)
            
            self.kNN = KNeighborsClassifier(n_neighbors=1)
            self.kNN.fit(sub_X,sub_y)
            classificadores.append(self.kNN)

        #classificadores = array de sample*3 len (9,15,21)

        if self.heuristica == "hillclimbing":
            #hiperparametros
            max_time = 120 #em segundos
            classificadores_optimal = hillclimbing_deterministico(max_time,classificadores,x_train,y_train,self._ordem_classes)
        elif self.heuristica == "simulatedannealing":
            #hiperparametros
            t = 200
            alfa = 0.1
            iter_max = 10
            max_time = 120 #em segundos
            classificadores_optimal = simulated_annealing(t,alfa,iter_max,max_time,classificadores,x_train,y_train,self._ordem_classes)
        elif self.heuristica == "genetic":
            #hiperparametros
            pop_size = 10
            max_iter = 200
            cross_ratio = 0.9
            mut_ratio = 0.1
            elite_pct = 20
            max_time = 120 #em segundos
            classificadores_optimal = genetic(pop_size,max_iter,cross_ratio,mut_ratio,elite_pct,max_time,classificadores,x_train,y_train,self._ordem_classes)
        else:
            print("default",self.heuristica)
            print("Método heuristico não encontrado.")

        #classificadores_optimal = hillclimbing_deterministico(classificadores,x_train,y_train,self._ordem_classes)

        #ESTE RETORNO VAI SER O OPTIMAL QUE O HILLCLIMBING RETORNAR
        self._classificadores = classificadores_optimal

    #método predict(): usado para predizer a classe dos exemplos do conjunto de teste
    def predict(self,x_test):
        ordem_classes = self._ordem_classes
        #print("len(self._classificadores)",len(self._classificadores))
        #predicts: vai conter uma lista com todos os predicts para cada classificador base
        predicts = []
        for classificador in self._classificadores:
            predict = classificador.predict(x_test)
            predicts.append(predict)
      
        dados_todos = pd.DataFrame(data=predicts)
        
        #predicao: array que será retornado pelo método predict
        #contendo cada uma das predição(para o hgpooling) para cada exemplo do conjunto de teste
        predicao = []
        
        for index in dados_todos: #loop para percorrer cada exemplo do conjunto de teste
            coluna = dados_todos[index] #coluna: todos os predicts dos classificadores base para um exemplo do teste
            
            #mostc: vai ser nx2 => na primeira coluna[0] vai ter a classe, na segunda[1] a quantidade
            #em ordem decrescente
            mostc = Counter(coluna).most_common() 

            #a condição a seguir cria um array empatados[]
            #para salvar os primeiros colocados entre 
            empatados = []
            if len(mostc) == 1: #caso só exista uma classe, não tem como ter empate, logo retorna ela mesma
                empatados.append(mostc[0][0])
            else: #caso tenha mais que uma classe, verifica se tem quantidades iguais
                for ind in range(len(mostc)-1):
                    if len(mostc) == 1:
                        empatados.append(mostc[ind][0])
                        break
                    else:
                        if mostc[ind][1] == mostc[ind+1][1]:
                            if empatados == []:    
                                empatados.append(mostc[ind][0])
                                empatados.append(mostc[ind+1][0])
                            else:
                                empatados.append(mostc[ind+1][0])
                        else:
                            if not empatados:
                                empatados.append(mostc[0][0])
                                break
                            else:
                                break
            #decisao: variavel auxiliar para armazenar o predict do atual exemplo
            #caso tenha mais de um no array empatados: vai usar o array ordem_classes para desempatar
            if len(empatados) != 1:                    
                for classe_ord in ordem_classes: #ordem_classes = array com a ordem para desempate obtido na base de treino original
                    for classe_pred in empatados: #empatados= array de classes empatadas
                        #o desempate é feito escolhendo entre os empatados o primeiro que aparecer na lista de ordem obitidos
                        if classe_ord == classe_pred: 
                            decisao = classe_pred
                            break
                    else:
                        continue
                    break
            else: #com apenas um, 
                decisao = empatados[0]
                    
            predicao.append(decisao)
            
        #predict = np.asarray(predicao)
        return np.asarray(predicao)


In [ ]:
def hillclimbing_deterministico(max_time,classificadores,x_test,y_train,ordem_classes):
        current_state = [0]*len(classificadores) #cria um state zerado [0,0,...,0] para usar como inicio
        optimal_acc = 0 #acuracia otima começa zerada junto com o state zerado
        optimal_state = current_state #estado otimo começa zerado junto com o state zerado
        valid_states = len(classificadores) #inicia valid_states, será usada como critério de parada da heuristica
        
        start = time.process_time() #start: tempo inicial
        end = 0                     #end:   tempo final

        #critérios de parada:
        #valid_states == 0 -> quando nenhum dos estados possiveis foi melhor que um estado de uma iteração anterior
        #end-start < max_time ->
        while valid_states != 0 and end-start <= max_time:
          #possible_states = todos os estados possiveis a partir do estado atual
          possible_states = generate_states(current_state)
          #valid_states = quantidade de estados possiveis
          valid_states = len(possible_states)

          #loop por todos os estados possiveis
          for state in possible_states:
              #aux_acc = acuracia do estado atual
              aux_acc = evaluate_state(state, classificadores,ordem_classes,x_test,y_train)
              
              #se acuracia atual for maior que a acuracia otima
              if aux_acc >= optimal_acc:
                  #atualiza a acuracia otima, o estado otimo, e o estado atual
                  optimal_acc = aux_acc
                  optimal_state = state
                  current_state = state
              else:
                  #senao, subtrai 1 do validstates (quando chega a 0 a heuristica para)
                  valid_states = valid_states - 1

          #end: tempo final
          end = time.process_time()

        #gen_state_comb = retorna os classificadores do estado otimo
        return gen_state_comb(optimal_state, classificadores)

In [ ]:
def simulated_annealing(t,alfa,iter_max,max_time,classificadores,x_test,y_train,ordem_classes):
    current_state = generate_initial_state(classificadores)
    optimal_state = current_state
    current_acc = evaluate_state(current_state, classificadores,ordem_classes,x_test,y_train)
    optimal_acc = current_acc

    start = time.process_time() #start: tempo inicial
    end = 0                     #end:   tempo final
    
    
    while t >= 1 and end-start <= max_time and optimal_acc != 1:        
        
        for _ in range(iter_max):
            neighborhood = generate_neighborhood(current_state)
            if neighborhood == []:
                return gen_state_comb(optimal_state, classificadores)

            aux_state = rand_state(neighborhood) #escolhe um state aleatorio dentro dos vizinhos
            aux_acc = evaluate_state(aux_state, classificadores,ordem_classes,x_test,y_train) #calcula acuracia aux_state

            if aux_acc > current_acc:
                current_state = aux_state
                current_acc = aux_acc
                if aux_acc > optimal_acc:
                    optimal_state = aux_state
                    optimal_acc = aux_acc
            else:
                if change_probability(aux_acc,current_acc,t):
                    current_state = aux_state
                    current_acc = aux_acc
        
        t = t*alfa
        end = time.process_time()

    return gen_state_comb(optimal_state, classificadores)

In [ ]:
def genetic(pop_size,max_iter,cross_ratio,mut_ratio,elite_pct,max_time,classificadores,x_test,y_train,ordem_classes):

    start = time.process_time()
    optimal_state = [0] * len(classificadores)
    optimal_acc = 0
    populacao = initial_population(pop_size, classificadores)
    conv = convergent(populacao)
    iter = 0    
    end = 0

    while not conv and iter < max_iter and end-start <= max_time and optimal_acc != 1:
        
        val_pop = evaluate_population(populacao, classificadores,ordem_classes,x_test,y_train)
        
        elitismo = elitism(val_pop, elite_pct)
        val_best = elitismo[0][0]

        new_pop = elitism_state(elitismo)
        best = new_pop[0]

        if (val_best > optimal_acc):
            optimal_state = best
            optimal_acc = val_best

        selected = selection(val_pop, pop_size - len(new_pop)) 
        crossed = crossover_step(selected, cross_ratio)
        mutated = mutation_step(crossed, mut_ratio)
        populacao = new_pop + mutated
        conv = convergent(populacao)
        iter+=1
        end = time.process_time()
        
    return gen_state_comb(optimal_state, classificadores)


In [ ]:
def predicao(predicts,ordem_classes):
  dados_todos = pd.DataFrame(data=predicts)
        
  #predicao: array que será retornado pelo método predict
  #contendo cada uma das predição(para o hgpooling) para cada exemplo do conjunto de teste
  predicao = []
  
  for index in dados_todos: #loop para percorrer cada exemplo do conjunto de teste
      coluna = dados_todos[index] #coluna: todos os predicts dos classificadores base para um exemplo do teste
      
      #mostc: vai ser nx2 => na primeira coluna[0] vai ter a classe, na segunda[1] a quantidade
      #em ordem decrescente
      mostc = Counter(coluna).most_common() 

      #a condição a seguir cria um array empatados[]
      #para salvar os primeiros colocados entre 
      empatados = []
      if len(mostc) == 1: #caso só exista uma classe, não tem como ter empate, logo retorna ela mesma
          empatados.append(mostc[0][0])
      else: #caso tenha mais que uma classe, verifica se tem quantidades iguais
          for ind in range(len(mostc)-1):
              if len(mostc) == 1:
                  empatados.append(mostc[ind][0])
                  break
              else:
                  if mostc[ind][1] == mostc[ind+1][1]:
                      if empatados == []:    
                          empatados.append(mostc[ind][0])
                          empatados.append(mostc[ind+1][0])
                      else:
                          empatados.append(mostc[ind+1][0])
                  else:
                      if not empatados:
                          empatados.append(mostc[0][0])
                          break
                      else:
                          break
      #decisao: variavel auxiliar para armazenar o predict do atual exemplo
      #caso tenha mais de um no array empatados: vai usar o array ordem_classes para desempatar
      if len(empatados) != 1:                    
          for classe_ord in ordem_classes: #ordem_classes = array com a ordem para desempate obtido na base de treino original
              for classe_pred in empatados: #empatados= array de classes empatadas
                  #o desempate é feito escolhendo entre os empatados o primeiro que aparecer na lista de ordem obitidos
                  if classe_ord == classe_pred: 
                      decisao = classe_pred
                      break
              else:
                  continue
              break
      else: #com apenas um, 
          decisao = empatados[0]
              
      predicao.append(decisao)
      
  #predict = np.asarray(predicao)
  #print("predicao:",predicao)
  return np.asarray(predicao)

In [ ]:
def evaluate_state(state, classificadores,ordem_classes,x_test,y_train):
    #comb_states = classificadores que estão no estado
    comb_states = gen_state_comb(state, classificadores)

    #predicts: vai conter todos os predicts para cada classificador
    predicts = []

    #loop por cada classificador que está no estado
    for classificador in comb_states:
        predict = classificador.predict(x_test)
        #calcula o predict do clasificador atual e salva em predicts
        predicts.append(predict)

    #predito: predicao vai escolher o mais votado entre os classificadores
    predito = predicao(predicts,ordem_classes)

    #calcula a acuracia do predito com o real
    value = accuracy_score(y_train, predito)
    
    return value

In [ ]:
def evaluate_population (populacao, classificadores,ordem_classes,x_test,y_train):
    #evaluate_population calcula a acuracia de cada estado e salva em um array com
    #[(acuracia,estado),(acuracia,estado),...,(acuracia,estado)]
    eval = []
    for state in populacao:
        eval = eval + [(evaluate_state(state, classificadores,ordem_classes,x_test,y_train), state)]
    return eval

In [ ]:
def generate_states(initial_state):
    #generate_states gera todos os estados possiveis a partir de um estado inicial(sempre 'pra frente')
    states = [] #array para salvar os estados gerados

    #for para cada elemento no estado inicial
    for i in range(len(initial_state)):
        aux = initial_state.copy() #aux = copia do estado inicial
        aux[i] = 1 #salva na posição i do estado copiado 1
        if(not np.array_equal(aux,initial_state)): #só salvar se o estado aux for diferente do estado inicial
          states.append(aux)
    #retorna o array de estados gerados
    return states

In [ ]:
def generate_initial_state(items):
    #generate_initial_state gera um estado inicial aleatorio do mesmo tamanho dos classificadores(items)
    zero_state = [0]*len(items) #cria um state todo de 0's para testar
    
    initial_state = [] #initial_state = cria o array que vai ser o estado inicial
    for i in range(len(items)): #for da quantidade de itens nos classificadores (9,15,21)
        initial_state.append(random.randint(0, 1)) #random de 0 ou 1, e adiciona no initial_state
    
    #o bloco abaixo é para não deixar o que o initial_state seja composto só de 0's
    while np.array_equal(zero_state,initial_state): #continua rodando até que seja difenrete do zero_state
        initial_state = []
        zero_state = [0]*len(items)
        for i in range(len(items)):
            initial_state.append(random.randint(0, 1))

    #retorna o initial_state com comprimento igual aos classificadores, composto de 0 e 1
    return initial_state

In [ ]:
def initial_population(n, items):
    #initial_population, gera uma população de estados de tamanho 'n' (n=hiperparametro do tamanho da população)
    pop = [] #pop = inicia o array que terá varios estados
    count = 0 #contador que irá até 'n'
    while count < n:
        individual = generate_initial_state(items) #individual = individuo gerado a partir do generate_initial state, que gera um estaod inicial aleatorio
        pop = pop + [individual] #insere o individual na populção
        count += 1 #contador++

    #retorna a população composto de 'n' individuos
    return pop

In [ ]:
def convergent(population):
    #convergent verifica se a população convergiu para um estado(todos os individuos serem iguais)
    conv = False
    if population != []:
        base = population[0] #base= pega o primeiro inviduo para servir de comparação
        i = 0
        while i < len(population):
            if base != population[i]: #compara com o primeiro individuo, caso diferente, a população nao ocnvergiu
                return False #retorna false para população com individuos diferentes
            i += 1
        return True #retorna true para população com todos os individuos iguais

In [ ]:
def state_size(state):
    #state_size = calcula o tamanho do estado, [1,1,1]= tamanho 3
    size = 0
    for i in range(len(state)):
        size += state[i]
    return size

In [ ]:
def change_state(state,position,value):
    #change_state, recebe um estado, uma posição, e um valor
    state[position] = value #e vai alterar o valor da posição do estado
    #retorna o estado alterado
    return state


def generate_neighborhood(state):
    #generate_neighborhood gera todos os estados vizinhos ao estado recebido
    neighborhood = [] #neighborhood= array para salvar os vizinhos
    zero_state = [0]*len(state) #cria um state todo de 0's para testar

    #dois for's, o primeiro vai passar inserindo 1 a cada posição do estado inicial
    #o segundo vai passar inserindo 0
    for i in range(len(state)):
        aux = state.copy()
        new_state = change_state(aux,i,1)
        if not np.array_equal(state,new_state): #caso o estado não seja igual ao inicial
            neighborhood.append(new_state) #salva o estado com a alteração no array de vizinhos
    for i in range(len(state)):
        aux = state.copy()
        new_state = change_state(aux,i,0)
        if not np.array_equal(state,new_state): #caso o estado não seja igual ao inicial
          if not np.array_equal(zero_state,new_state): # e caso, não seja [0,0,0] só de 0's
            neighborhood.append(new_state) #salva o estado com a alteração no array de vizinhos
    #retorna todos os vizinhos pelo array neighborhood
    return neighborhood

In [ ]:
def rand_state(states):
    #rand_state = escolhe um estado aleatorio, a partir de um array de estados(states)
    index = random.randint(0,len(states)-1) #gera um index aleatorio
    #retorna o estado do index gerado do states
    return states[index]

In [ ]:
def change_probability(value,best_value,t):
    p = 1/(math.exp(1)**((best_value-value)/t))
    r = random.uniform(0,1)
    if r < p:
        return True
    else:
        return False

In [ ]:
def gen_state_comb(state, items):
    combinado_atual = []
    for i in range(len(state)):
        if(state[i] == 1):
          combinado_atual.append(items[i])
    return combinado_atual

In [ ]:
def first(x):
    return x[0]


def elitism (val_pop, pct):
    n = math.floor((pct/100)*len(val_pop))
    if n < 1:
        n = 1
    val_elite = sorted (val_pop, key = first, reverse = True)[:n]
    #elite = [s for v,s in val_elite]
    return val_elite

def elitism_state(val_elite):
    elite = [s for v,s in val_elite]
    return elite

#valor = elitism ([(6, [1, 0, 1]), (9, [0, 1, 1]), (5, [0, 0, 1])], 70)

#print(valor[0][0])

#print(elitism_state([(6, [1, 0, 1]), (9, [0, 1, 1]), (5, [0, 0, 1])]))


In [ ]:
def crossover(dad,mom):
    zero_state = [0]*len(dad)
    r = random.randint(1, len(dad) - 1)
    son = dad[:r]+mom[r:]
    daug = mom[:r]+dad[r:]

    while np.array_equal(zero_state,son) or np.array_equal(zero_state,daug):
        r = random.randint(1, len(dad) - 1)
        son = dad[:r]+mom[r:]
        daug = mom[:r]+dad[r:]

    return son, daug



def crossover_step (population, crossover_ratio):
    new_pop = []

    if len(population) < 2:
        return population
    else:
        for _ in range (round(len(population)/2)):
            rand = random.uniform(0, 1)
            fst_ind = random.randint(0, len(population) - 1)
            scd_ind = fst_ind
            while fst_ind == scd_ind:
              scd_ind = random.randint(0, len(population) - 1)
            parent1 = population[fst_ind]
            parent2 = population[scd_ind]

            if rand <= crossover_ratio:
                offspring1, offspring2 = crossover(parent1, parent2)            
            else:
                offspring1, offspring2 = parent1, parent2

            new_pop = new_pop + [offspring1, offspring2]
        
    return new_pop


#print(crossover([0,0,0,1,0,0],[0,1,0,0,0,0]))
#print(crossover_step ([[0,0,0,0,0,0],[1,1,1,1,1,1]], 0.5))

In [ ]:
def mutation (indiv):
    zero_state = [0]*len(indiv)

    individual = indiv.copy()
    rand = random.randint(0, len(individual) - 1)
    if individual[rand] == 0:
        individual[rand] = 1
    else:
        individual[rand] = 0
    
    while np.array_equal(zero_state,individual):
        individual = indiv.copy()
        rand = random.randint(0, len(individual) - 1)
        if individual[rand] == 0:
            individual[rand] = 1
        else:
            individual[rand] = 0

    return individual



def mutation_step (population, mutation_ratio):
    ind = 0
    for individual in population:
        rand = random.uniform(0, 1)

        if rand <= mutation_ratio:
            mutated = mutation(individual)
            population[ind] = mutated
                
        ind+=1
        
    return population


#mutation ([0,0,0,0,0,1,0,0,0])
#mutation_step ([[1, 1, 1], [1, 1, 0], [1, 0, 0], [0, 0, 0]], 0.8)

In [ ]:
def states_total_value(states):
    total_sum = 0
    for state in states:
        total_sum = total_sum + state[0]
    return total_sum


def roulette_construction(states):
    aux_states = []
    roulette = []
    total_value = states_total_value(states)
    for state in states:
        value = state[0]
        if total_value != 0:
            ratio = value/total_value
        else:
            ratio = 1
        aux_states.append((ratio,state[1]))
 
    acc_value = 0
    for state in aux_states:
        acc_value = acc_value + state[0]
        s = (acc_value,state[1])
        roulette.append(s)
    return roulette

def roulette_run (rounds, roulette):
    if roulette == []:
        return []
    selected = []
    while len(selected) < rounds:
        r = random.uniform(0,1)
        for state in roulette:
            if r <= state[0]:
                selected.append(state[1])
                break
    return selected


def selection(value_population,n):
    aux_population = roulette_construction(value_population)
    new_population = roulette_run(n, aux_population)
    return new_population

#roulette_construction([(1, [1,0,0,0,0,0]), (0.9, [0,1,0,0,0,0]), (1, [0,0,1,0,0,0])])
#roulette_run (1,[(0.3448275862068966, [1, 0, 0, 0, 0, 0]),(0.6551724137931034, [0, 1, 0, 0, 0, 0]),(1.0, [0, 0, 1, 0, 0, 0])])
#selection([(1, [1,0,0,0,0,0]), (0.9, [0,1,0,0,0,0]), (1, [0,0,1,0,0,0])],2)

In [ ]:
#função para calcular a média das acuracias, desvio padrão, intervalo a 95% inferior e superior
def scor(scores):
    mean = scores.mean()
    std = scores.std()
    inf, sup = stats.norm.interval(0.95, loc=mean,
                               scale=std/np.sqrt(len(scores)))
    return mean,std,inf,sup

In [ ]:
stdscaler = preprocessing.StandardScaler() #normalizacao z-score
grade_hgPooling={'estimator__n_samples': [3,5,7]} #a grade para o ciclo intenro do hgpooling é diferente, passando n_samples
rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3,random_state=36851234) #ciclo externo

In [ ]:
#hillclimbing
hgPooling_hc = HeterogenousPoolingClassifier_Heuristico('hillclimbing')
pipe_hgPooling_hc = Pipeline([('scaler', stdscaler), ('estimator', hgPooling_hc)])

gs_hgPooling_hc = GridSearchCV(estimator=pipe_hgPooling_hc, param_grid = grade_hgPooling, scoring='accuracy', cv = 4)

scores_wine_hgPooling_hc = cross_val_score(gs_hgPooling_hc, wine_X, wine_y, scoring='accuracy', cv = rkf)

print(scores_wine_hgPooling_hc)
print(scor(scores_wine_hgPooling_hc))

[1.         1.         0.94444444 1.         0.88888889 1.
 1.         0.94444444 1.         0.88235294 1.         1.
 1.         1.         1.         0.88888889 0.94444444 1.
 1.         0.94117647 0.94444444 1.         1.         0.94444444
 1.         1.         1.         0.94444444 0.94117647 1.        ]
(0.9736383442265795, 0.03791350479027273, 0.9600714200788698, 0.9872052683742891)


In [ ]:
#simulatedannealing
hgPooling_sa = HeterogenousPoolingClassifier_Heuristico('simulatedannealing')
pipe_hgPooling_sa = Pipeline([('scaler', stdscaler), ('estimator', hgPooling_sa)])

gs_hgPooling_sa = GridSearchCV(estimator=pipe_hgPooling_sa, param_grid = grade_hgPooling, scoring='accuracy', cv = 4)

scores_wine_hgPooling_sa = cross_val_score(gs_hgPooling_sa, wine_X, wine_y, scoring='accuracy', cv = rkf)

print(scores_wine_hgPooling_sa)
print(scor(scores_wine_hgPooling_sa))

In [ ]:
#genetic
hgPooling_ge = HeterogenousPoolingClassifier_Heuristico('genetic')
pipe_hgPooling_ge = Pipeline([('scaler', stdscaler), ('estimator', hgPooling_ge)])

gs_hgPooling_ge = GridSearchCV(estimator=pipe_hgPooling_ge, param_grid = grade_hgPooling, scoring='accuracy', cv = 4)

scores_wine_hgPooling_ge = cross_val_score(gs_hgPooling_ge, wine_X, wine_y, scoring='accuracy', cv = rkf)

print(scores_wine_hgPooling_ge)
print(scor(scores_wine_hgPooling_ge))

[1.         1.         0.94444444 1.         0.88888889 1.
 1.         1.         1.         0.88235294 0.94444444 1.
 1.         1.         1.         0.88888889 1.         0.94444444
 1.         0.94117647 0.94444444 1.         0.94444444 0.94444444
 1.         1.         1.         0.94444444 1.         1.        ]
(0.9737472766884531, 0.03782467004500326, 0.9602121410619423, 0.9872824123149638)
